# 3-1 선형 회귀와 자동 미분
## 1. 훈련 데이터셋의 구성
- 학습 데이터는 파이토치의 텐서의 형태를 가져야한다.
- 입력과 출력을 각기 다른 텐서에 저장할 필요가 있다.
- `x_train = torch.FloatTensor([[1], [2], [3]])`
- `y_train = torch.FloatTensor([[2], [4], [6]])`
- 여기서 x_train은 공부한 시간, y_train은 그에 맵핑되는 점수를 의미한다.

## 2. 가설(Hypothesis) 수립
- 머신 러닝에서 식을 세울 때 이 식을 가설이라고 한다.
- 보통 머신 러닝에서 가설은 임의로 추측해서 세워보는 식일수도 있고, 경험적으로 알고 있는 식일 수도 있다.
- 맞는 가설이 아니라고 판단되면 계속 수정해나가게 되는 식이기도 하다.
- 선형 회귀란 학습 데이터와 가장 잘 맞는 하나의 직선을 찾는 일이다.
- 이 때 선형 회귀의 가설은 **y = Wx + b**와 같은 형태이다.
- **H(x) = Wx + b**처럼 표현하기도 한다.
- 이때 x와 곱해지는 **W**를 가중치라고 하며, b를 편향이라고 한다.

## 3. 비용 함수에 대한 이해
- **비용 함수(cost function) = 손실 함수(loss function) = 오차 함수(error function) = 목적 함수(objective function)**
- MSE(Mean Squared Error) : $\frac{1}{n} \sum_{i=0}^{n}[y^i - H(x^i)]^2$
- 평균 제곱 오차는 회귀 문제에서 적절한 **W**와 b를 찾기위해서 최적화된 식이다.
- 평균 제곱 오차의 값을 최소값으로 만드는 **W**와 b를 찾아내는 것이 가장 훈련 데이터를 잘 반영한 직선을 찾아내는 일이기 때문이다.
- Cost(W, b) = $\frac{1}{n} \sum_{i=0}^{n}[y^i - H(x^i)]^2$
- Cost(W, b)를 최소가 되게 만드는 W와 b를 구하면 훈련 데이터를 가장 잘 나타내는 직선을 구할 수 있다.

## 4. 옵티마이저 - 경사 하강법
- cost가 최소화가 되는 지점은 접선의 기울기가 0이 되는 지점이며, 또한 미분값이 0이 되는 지점이다.
- 경사 하강법의 아이디어는 비용 함수를 미분하여 현재 **W**에서의 접선의 기울기를 구하고, 접선의 기울기가 낮은 방향으로 **W**의 값을 변경하는 작업을 반복하는 것에 있다.
- 현재 **W**에 접선의 기울기를 구해 특정 숫자 α를 곱한 값을 빼서 새로운 **W**로 사용하는 식이 사용된다.
- gradient = $\frac{∂cost(W)}{∂W}$
- **기울기가 음수일 때 : W의 값이 증가**
  - $W := W - α\times (-gradient) = W + α\times gradient$
  - 기울기가 음수면 **W**의 값이 증가하는데 이는 결과적으로 접선의 기울기가 0인 방향으로 **W**의 값이 조정된다.
- **기울기가 양수일 때 : W의 값이 감소**
  - $W := W - α\times (+gradient)$
  - 기울기가 양수면 **W**의 값이 감소하게 되는 이는 결과적으로 기울기가 0인 방향으로 **W**의 값이 조정된다.
- $W := W - α\times \frac{∂}{∂W}cost(W)$
  - 이 수식은 접선의 기울기가음수거나, 양수일 때 모두 접선의 기울기가 0인 방향으로 **W**의 값을 조정한다.
- 학습률 α는 **W**의 값을 변경할 때, 얼마나 크게 변경할지를 결정한다.
- 학습률이 지나치게 클 경우 접선의 기울기가 0이 되는 **W**를 찾아가는 것이 아니라 cost의 값이 발산한다.
- 지나치게 낮은 값을 가지면 학습 속도가 느려지므로 적당한 α의 값을 찾아내는 것도 중요하다.


In [ ]:
# 파이토치로 선형 회귀 구현하기
# 1) 기본 셋팅
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 재실행해도 같은 결과가 나오도록 랜덤 시드 설정
torch.manual_seed(1)

In [ ]:
# 2) 변수 선언
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

# 각 데이터 출력 및 크기 출력
print(x_train)
print(x_train.shape)
print(y_train)
print(y_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])
tensor([[2.],
        [4.],
        [6.]])
torch.Size([3, 1])


In [ ]:
# 3) 가중치와 편향의 초기화
# 가중치 W를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시함.
# requires_grad=True -> 학습을 통해 계속 값이 변경되는 변수임을 의미
W = torch.zeros(1, requires_grad=True)
print(W)
b = torch.zeros(1, requires_grad=True)
print(b)

tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)


In [ ]:
# 4) 가설 세우기
# H(x) = Wx + b
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


In [ ]:
# 5) 비용 함수 선언하기
# 평균 제곱 오차 선언
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(18.6667, grad_fn=<MeanBackward0>)


In [ ]:
# 6) 경사 하강법 구현하기
optimizer = optim.SGD([W, b], lr=0.01)
# optimizer.zero_grad()를 실행하므로서 미분을 통해얻은 기울기를 0으로 초기화
'''
보통 딥러닝에서는 미니배치 + 루프 조합을 사용해서 parameter들을 업데이트 하는데,
한 루프에서 업데이트를 위해 loss.backward()를 호출하면 각 파라미터들의 .grad 값에 변화도가 저장된다.
이후 다음 루프에서 zero_grad()를 하지 않고 역전파를 시키면 이전 루프에서 .grad에 저장된 값이
다음 루프의 업데이트에도 간섭을 해서 원하는 방향으로 학습이 되지 않는다.
따라서 루프가 한 번 돌고나서 역전파를 하기전에 반드시 zero_grad()로 .grad 값들을 0으로 초기화시킨 후 학습을 진행해야한다.
'''
# cost.backward() 함수를 호출하면 가중치 W와 편향 b에 대한 기울기가 계산된다.
# optimizer.step() 함수를 호출하여 인수로 들어갔던 W와 b에서 리턴되는 변수들의 기울기에 학습률 0.01을 곱하여 빼줌으로서 업데이트한다.

'\n보통 딥러닝에서는 미니배치 + 루프 조합을 사용해서 parameter들을 업데이트 하는데,\n한 루프에서 업데이트를 위해 loss.backward()를 호출하면 각 파라미터들의 .grad 값에 변화도가 저장된다.\n이후 다음 루프에서 zero_grad()를 하지 않고 역전파를 시키면 이전 루프에서 .grad에 저장된 값이\n다음 루프의 업데이트에도 간섭을 해서 원하는 방향으로 학습이 되지 않는다.\n따라서 루프가 한 번 돌고나서 역전파를 하기전에 반드시 zero_grad()로 .grad 값들을 0으로 초기화시킨 후 학습을 진행해야한다.\n'

In [ ]:
# 7) 전체 코드
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1999 # 원하는만큼 경사 하강법을 반복
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  hypothesis = x_train * W + b

  # cost 계산
  cost = torch.mean((hypothesis - y_train) ** 2)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
        epoch, nb_epochs, W.item(), b.item(), cost.item()
    ))

Epoch    0/1999 W: 0.187, b: 0.080 Cost: 18.666666
Epoch  100/1999 W: 1.746, b: 0.578 Cost: 0.048171
Epoch  200/1999 W: 1.800, b: 0.454 Cost: 0.029767
Epoch  300/1999 W: 1.843, b: 0.357 Cost: 0.018394
Epoch  400/1999 W: 1.876, b: 0.281 Cost: 0.011366
Epoch  500/1999 W: 1.903, b: 0.221 Cost: 0.007024
Epoch  600/1999 W: 1.924, b: 0.174 Cost: 0.004340
Epoch  700/1999 W: 1.940, b: 0.136 Cost: 0.002682
Epoch  800/1999 W: 1.953, b: 0.107 Cost: 0.001657
Epoch  900/1999 W: 1.963, b: 0.084 Cost: 0.001024
Epoch 1000/1999 W: 1.971, b: 0.066 Cost: 0.000633
Epoch 1100/1999 W: 1.977, b: 0.052 Cost: 0.000391
Epoch 1200/1999 W: 1.982, b: 0.041 Cost: 0.000242
Epoch 1300/1999 W: 1.986, b: 0.032 Cost: 0.000149
Epoch 1400/1999 W: 1.989, b: 0.025 Cost: 0.000092
Epoch 1500/1999 W: 1.991, b: 0.020 Cost: 0.000057
Epoch 1600/1999 W: 1.993, b: 0.016 Cost: 0.000035
Epoch 1700/1999 W: 1.995, b: 0.012 Cost: 0.000022
Epoch 1800/1999 W: 1.996, b: 0.010 Cost: 0.000013
Epoch 1900/1999 W: 1.997, b: 0.008 Cost: 0.000008

In [ ]:
# 6) optimizer.zero_grad()가 필요한 이유
# 파이토치는 미분을 통해 얻은 기울기를 이전에 계산된 기울기 값에 누적시키는 특징이 있다.

w = torch.tensor(2.0, requires_grad=True)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
  z = 2 * w

  z.backward()
  print('수식을 w로 미분한 값 : {}'.format(w.grad))

수식을 w로 미분한 값 : 2.0
수식을 w로 미분한 값 : 4.0
수식을 w로 미분한 값 : 6.0
수식을 w로 미분한 값 : 8.0
수식을 w로 미분한 값 : 10.0
수식을 w로 미분한 값 : 12.0
수식을 w로 미분한 값 : 14.0
수식을 w로 미분한 값 : 16.0
수식을 w로 미분한 값 : 18.0
수식을 w로 미분한 값 : 20.0
수식을 w로 미분한 값 : 22.0
수식을 w로 미분한 값 : 24.0
수식을 w로 미분한 값 : 26.0
수식을 w로 미분한 값 : 28.0
수식을 w로 미분한 값 : 30.0
수식을 w로 미분한 값 : 32.0
수식을 w로 미분한 값 : 34.0
수식을 w로 미분한 값 : 36.0
수식을 w로 미분한 값 : 38.0
수식을 w로 미분한 값 : 40.0
수식을 w로 미분한 값 : 42.0


In [ ]:
'''
텐서에는 requires_grad라는 속성이 있는데 이것을 True로 설정하면 자동 미분 기능이 적용된다.
선형 회귀부터 신경망과 같은 복잡한 구조에서 파라미터들이 모두 이 기능이 적용된다.
requires_grad = True가 적용된 텐서에 연산을 하면, 계산 그래프가 생성되며 backward 함수를 호출하면
그래프로부터 자동으로 미분이 계산된다.
'''

In [ ]:
# 8) 자동 미분 실습하기
# 값이 2인 임의의 스칼라 텐서 w 선언
# requires_grad=True로 설정하면 이 텐서에 대한 기울기를 저장하겠다는 의미이다.
w = torch.tensor(2.0, requires_grad=True)
y = w ** 2
z = 2 * y + 5
z.backward()
print(f'수식을 w로 미분한 값 : {w.grad}')

수식을 w로 미분한 값 : 8.0


# 3-2 다중 선형 회귀
$H(x) = w_1x_1 + w_2x_2 + w_3x_3 + b$

In [3]:
# 파이토치로 구현하기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# 훈련 데이터
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # H(x) 계산
  hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

  # cost 계산
  cost = torch.mean((hypothesis - y_train) ** 2)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
        epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
    ))

Epoch    0/1000 w1: 0.294 w2: 0.294 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.661 w3: 0.676 b: 0.008 Cost: 1.563628
Epoch  200/1000 w1: 0.679 w2: 0.655 w3: 0.677 b: 0.008 Cost: 1.497595
Epoch  300/1000 w1: 0.684 w2: 0.649 w3: 0.677 b: 0.008 Cost: 1.435044
Epoch  400/1000 w1: 0.689 w2: 0.643 w3: 0.678 b: 0.008 Cost: 1.375726
Epoch  500/1000 w1: 0.694 w2: 0.638 w3: 0.678 b: 0.009 Cost: 1.319507
Epoch  600/1000 w1: 0.699 w2: 0.633 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.627 w3: 0.679 b: 0.009 Cost: 1.215703
Epoch  800/1000 w1: 0.709 w2: 0.622 w3: 0.679 b: 0.009 Cost: 1.167810
Epoch  900/1000 w1: 0.713 w2: 0.617 w3: 0.680 b: 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2: 0.613 w3: 0.680 b: 0.009 Cost: 1.079390


In [4]:
# 벡터와 행렬 연산으로 바꾸기
# 벡터와 행렬 연산은 식을 간단하게 해줄 뿐만 아니라 다수의 샘플의 병렬 연산이므로 속도의 이점을 가짐
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 80],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# x_train과 y_train의 크기 출력
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [5]:
# 가중치와 편향 선언
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  hypothesis = x_train.matmul(W) + b

  # cost 계산
  cost = torch.mean((hypothesis - y_train) ** 2)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([66.7178, 80.1701, 76.1025, 86.0194, 61.1565]) Cost: 9537.694336
Epoch    2/20 hypothesis: tensor([104.5421, 125.6208, 119.2478, 134.7862,  95.8280]) Cost: 3069.590088
Epoch    3/20 hypothesis: tensor([125.9858, 151.3882, 143.7087, 162.4333, 115.4844]) Cost: 990.670288
Epoch    4/20 hypothesis: tensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]) Cost: 322.481873
Epoch    5/20 hypothesis: tensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]) Cost: 107.717064
Epoch    6/20 hypothesis: tensor([148.9423, 178.9730, 169.8976, 192.0301, 136.5279]) Cost: 38.687496
Epoch    7/20 hypothesis: tensor([151.1574, 181.6346, 172.4254, 194.8856, 138.5585]) Cost: 16.499043
Epoch    8/20 hypothesis: tensor([152.4131, 183.1435, 173.8590, 196.5043, 139.7097]) Cost: 9.365656
Epoch    9/20 hypothesis: tensor([153.1250, 183.9988, 174.6723, 197.4217, 140.3625]) Cost: 7.071114
Epoch   10/20 hyp

In [9]:
# 임의의 값에 대한 예측
# with torch.no_grad() : 이 블록 안에서 수행되는 모든 연산에 대해 역전파를 비활성화.
# 예측을 할 때는 가중치를 업데이트할 필요가 없기 때문에, 메모리와 계산 자원을 절약하기 위해 torch.no_grad()를 사용하는 것이 좋음
with torch.no_grad():
  new_input = torch.FloatTensor([[75, 85, 72]]) # 예측하고 싶은 임의의 입력
  prediction = new_input.matmul(W) + b
  print('Predicted value for input {}: {}'.format(new_input.squeeze().tolist(), prediction.item()))

Predicted value for input [75.0, 85.0, 72.0]: 156.8051300048828


# 3-3 nn.Module과 클래스로 구현하기
## 파이토치에 구현된 함수 사용
- 선형회귀 모델은 nn.Linear()라는 함수로 구현되어 있음
- 평균 제곱 오차는 nn.functional.mse_loss()라는 함수로 구현되어 있음

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
model = nn.Linear(1, 1)

# model.parameters()라는 함수를 사용하면 가중치 W와 편향 b를 불러올 수 있음
# 첫 번째 값이 W, 두 번째 값이 b
print(list(model.parameters()))

# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 전체 훈련 데이터에 대해 경사 하강법을 2,000회 반복
nb_epochs = 2000
for epoch in range(nb_epochs+1):
  # H(x) 계산
  prediction = model(x_train)

  # cost 계산
  cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

  # cost로 H(x) 개선하는 부분
  optimizer.zero_grad() # gradient를 0으로 초기화
  cost.backward() # 비용 함수를 미분하여 gradient 계산
  optimizer.step() # W와 b를 업데이트

  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]
Epoch    0/2000 Cost: 13.103541
Epoch  100/2000 Cost: 0.002791
Epoch  200/2000 Cost: 0.001724
Epoch  300/2000 Cost: 0.001066
Epoch  400/2000 Cost: 0.000658
Epoch  500/2000 Cost: 0.000407
Epoch  600/2000 Cost: 0.000251
Epoch  700/2000 Cost: 0.000155
Epoch  800/2000 Cost: 0.000096
Epoch  900/2000 Cost: 0.000059
Epoch 1000/2000 Cost: 0.000037
Epoch 1100/2000 Cost: 0.000023
Epoch 1200/2000 Cost: 0.000014
Epoch 1300/2000 Cost: 0.000009
Epoch 1400/2000 Cost: 0.000005
Epoch 1500/2000 Cost: 0.000003
Epoch 1600/2000 Cost: 0.000002
Epoch 1700/2000 Cost: 0.000001
Epoch 1800/2000 Cost: 0.000001
Epoch 1900/2000 Cost: 0.000000
Epoch 2000/2000 Cost: 0.000000


In [14]:
# 임의의 입력 4를 선언
new_var = torch.FloatTensor([[4.0]])
# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
print('훈련 후 입력이 4일 때의 예측값 :', pred_y)

훈련 후 입력이 4일 때의 예측값 : tensor([[7.9989]], grad_fn=<AddmmBackward0>)


In [15]:
# 학습 후의 W와 b의 값
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9994]], requires_grad=True), Parameter containing:
tensor([0.0014], requires_grad=True)]


In [17]:
# 다중 선형 회귀 구현
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 선언 및 초기화. 다중 선형 회귀이므로 input_dim=3, output_dim=1
model = nn.Linear(3, 1)

# 가중치와 편향 출력
print(list(model.parameters()))

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 2000
for epoch in range(nb_epochs + 1):

  # H(x) 계산
  prediction = model(x_train)
  # model(x_train)은 model.forward(x_train)와 동일함.

  # cost 계산
  cost = F.mse_loss(prediction, y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

[Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119]], requires_grad=True), Parameter containing:
tensor([0.2710], requires_grad=True)]
Epoch    0/2000 Cost: 31667.597656
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210560
Epoch 1100/2000 Cost: 0.209232
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206761
Epoch 1400/2000 Cost: 0.205619
Epoch 1500/2000 Cost: 0.204522
Epoch 1600/2000 Cost: 0.203484
Epoch 1700/2000 Cost: 0.202485
Epoch 1800/2000 Cost: 0.201542
Epoch 1900/2000 Cost: 0.200635
Epoch 2000/2000 Cost: 0.199769


In [18]:
# 임의의 입력 [73, 80, 75]를 선언
new_var = torch.FloatTensor([[73, 80, 75]])
# 입력한 값에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var)
print('훈련 후 입력이 73, 80, 75일 때의 예측값 :', pred_y)

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[151.2305]], grad_fn=<AddmmBackward0>)


In [19]:
# w와 b값을 출력
print(list(model.parameters()))

[Parameter containing:
tensor([[0.9778, 0.4539, 0.5768]], requires_grad=True), Parameter containing:
tensor([0.2802], requires_grad=True)]


In [20]:
# 모델을 클래스로 구현
# 클래스를 사용한 모델 구현 형식은 대부분의 파이토치 구현체에서 사용하고 있는 방식
class LinearRegressionModel(nn.Module): # torch.nn.Module을 상속받은 파이썬 클래스
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(1, 1) # 단순 선형 회귀

    def forward(self, x):
      return self.linear(x)

model = LinearRegressionModel()

# 다중 선형 회귀를 클래스로 구현
class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1) # 다중 선형 회귀

  def forward(self, x):
    return self.linear(x)

model = MultivariateLinearRegressionModel()

In [21]:
# 단순 선형 회귀 클래스로 구현하기
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [7]])

class LinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(1, 1)

  def forward(self, x):
    return self.linear(x)

model = LinearRegressionModel()

# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# 전체 훈련 데이터에 대해 경사 하강법을 2,000회 반복
nb_epochs = 2000
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  prediction = model(x_train)

  # cost 계산
  cost = F.mse_loss(prediction, y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/2000 Cost: 16.700600
Epoch  100/2000 Cost: 0.130634
Epoch  200/2000 Cost: 0.101949
Epoch  300/2000 Cost: 0.084224
Epoch  400/2000 Cost: 0.073271
Epoch  500/2000 Cost: 0.066503
Epoch  600/2000 Cost: 0.062320
Epoch  700/2000 Cost: 0.059736
Epoch  800/2000 Cost: 0.058139
Epoch  900/2000 Cost: 0.057152
Epoch 1000/2000 Cost: 0.056542
Epoch 1100/2000 Cost: 0.056165
Epoch 1200/2000 Cost: 0.055932
Epoch 1300/2000 Cost: 0.055788
Epoch 1400/2000 Cost: 0.055699
Epoch 1500/2000 Cost: 0.055644
Epoch 1600/2000 Cost: 0.055611
Epoch 1700/2000 Cost: 0.055589
Epoch 1800/2000 Cost: 0.055576
Epoch 1900/2000 Cost: 0.055569
Epoch 2000/2000 Cost: 0.055564


In [22]:
# 다중 선형 회귀 클래스로 구현
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 2000
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  prediction = model(x_train)

  # cost 계산
  cost = F.mse_loss(prediction, y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/2000 Cost: 31667.597656
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210560
Epoch 1100/2000 Cost: 0.209232
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206761
Epoch 1400/2000 Cost: 0.205619
Epoch 1500/2000 Cost: 0.204522
Epoch 1600/2000 Cost: 0.203484
Epoch 1700/2000 Cost: 0.202485
Epoch 1800/2000 Cost: 0.201542
Epoch 1900/2000 Cost: 0.200635
Epoch 2000/2000 Cost: 0.199769


# 3-4 미니 배치와 데이터 로더
## 1. 미니 배치와 배치 크기
- 전체 데이터에 대해서 경사 하강법을 수행하는 것은 매우 느릴 뿐만 아니라 많은 계산량이 필요
- 메모리의 한계로 계산이 불가능한 경우도 있을 수 있음
- 전체 데이터를 더 작은 단위로 나누어서 학습하는 개념이 나옴
- 이 단위를 미니배치라고 한다.
- 미니 배치 학습을 하게 되면 미니 배치만큼만 가져가서 미니 배치에 대한 비용을 계산하고 경사 하강법을 수행한다.
- 다음 미니 배치를 가져가서 경사 하강법을 수행하고 마지막미니 배치까지 반복하여 전체 데이터에 대한 학습이 1회 끝나면 1 에포크가 끝난 것이다
- 미니 배치의 개수는 미니 배치의 크기를 몇으로 하느냐에 따라서 달라지는데 미니 배치의 크기를 배치 크기라고 한다.
- 전체 데이터에 대해서 한 번에 경사 하강법을 수행하는 방법을 배치 **경사 하강법**이라고 부른다.
- 미니 배치 단위로 경사 하강법을 수행하는 방법을 **미니 배치 경사 하강법**이라고 부른다.
- 배치 경사 하강법은 경사 하강법을 할 때, 전체 데이터를 사용하므로 가중치 값이 최적값에 수렴하는 과정이 매우 안정적이지만, 계산량이 너무 많음.
- 미니 배치 경사 하강법은 경사 하강법을 할 때, 전체 데이터의 일부만을 수행하므로 최적값으로 수렴하는 과정에서 값이 조금 헤매기도 하지만 훈련 속도가 빠르다.
- 배치 크기는 보통 2의 제곱수를 사용한다.
  - CPU와 GPU의 메모리가 2의 배수이므로 배치크기가 2의 제곱수일 경우에 데이터의 송수신 효율을 높일 수 있음.
  
## 2. 이터레이션
- 이터레이션은 한 번의 에포크 내에서 이루어지는 매개변수인 가중치 W와 b의 업데이트 횟수이다.
- 전체 데이터가 2000일 때 배치 크기를 200으로 한다면 이터레이션의 수는 총 10개이다.

In [26]:
# 데이터 로드하기
# 데이터를 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 데이터셋과 데이터로더를 제공
# 이를 사용하면 미니 배치 학습, 데이터 셔플, 병렬 처리까지 간단하게 수행할 수 있다.
# 기본적인 사용법은 Dataset을 정의하고 이를 DataLoader에 전달하는 것이다.

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset # 텐서데이터셋. 텐서를 입력받아 Dataset의 형태로 변환해줌
from torch.utils.data import DataLoader # 데이터로더

x_train  =  torch.FloatTensor([[73,  80,  75],
                               [93,  88,  93],
                               [89,  91,  90],
                               [96,  98,  100],
                               [73,  66,  70]])
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

dataset = TensorDataset(x_train, y_train) # datset으로 저장

# 데이터로더는 기본적으로 데이터셋과 미니 배치의 크기를 인자로 입력받는다.
# shuffle=True를 선택하면 Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꾼다.(권장됨)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

model = nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 50
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx + 1, len(dataloader), cost.item()
    ))

Epoch    0/50 Batch 1/3 Cost: 19805.816406
Epoch    0/50 Batch 2/3 Cost: 3113.049805
Epoch    0/50 Batch 3/3 Cost: 1298.395020
Epoch    1/50 Batch 1/3 Cost: 608.364197
Epoch    1/50 Batch 2/3 Cost: 110.540558
Epoch    1/50 Batch 3/3 Cost: 32.827255
Epoch    2/50 Batch 1/3 Cost: 10.116283
Epoch    2/50 Batch 2/3 Cost: 18.584768
Epoch    2/50 Batch 3/3 Cost: 3.204240
Epoch    3/50 Batch 1/3 Cost: 0.509706
Epoch    3/50 Batch 2/3 Cost: 8.668556
Epoch    3/50 Batch 3/3 Cost: 0.275426
Epoch    4/50 Batch 1/3 Cost: 1.903194
Epoch    4/50 Batch 2/3 Cost: 0.914394
Epoch    4/50 Batch 3/3 Cost: 14.852406
Epoch    5/50 Batch 1/3 Cost: 3.888444
Epoch    5/50 Batch 2/3 Cost: 6.443260
Epoch    5/50 Batch 3/3 Cost: 2.056635
Epoch    6/50 Batch 1/3 Cost: 1.237848
Epoch    6/50 Batch 2/3 Cost: 8.351044
Epoch    6/50 Batch 3/3 Cost: 0.162975
Epoch    7/50 Batch 1/3 Cost: 6.530087
Epoch    7/50 Batch 2/3 Cost: 3.324821
Epoch    7/50 Batch 3/3 Cost: 0.528692
Epoch    8/50 Batch 1/3 Cost: 0.379898
Epoch  

In [28]:
# 임의의 입력 [73, 80, 75]를 선언
new_var = torch.FloatTensor([[73, 80, 75]])
# 임력한 값에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var)
print('훈련 후 입력이 73, 80, 75일 때의 예측값 :', pred_y.item())

훈련 후 입력이 73, 80, 75일 때의 예측값 : 148.00003051757812


# 커스템 데이터셋
- torch.utils.data.Dataset을 상속받아 직접 커스템 데이터셋을 만드는 경우도 있다.
- torch.utils.data.Dataset은 데이터셋을 제공하는 추상 클래스이다.
- Dataset을 상속받아 __init__, __len__, __getitem__을 오버라이드 하여 커스템 데이터셋을 만들 수 있음.
- `__init__` : 데이터셋의 전처리를 해주는 부분
- `__len__` : 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
- `__ getitem__` : 데이터셋에서 특정 1개의 샘플을 가져오는 함수
- len(dataset)을 했을 때 데이터셋의 크기를 리턴할 **len**
- dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 **get_item**

In [29]:
# 커스텀 데이터셋으로 선형 회귀 구현하기
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Dataset 상속
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx + 1, len(dataloader), cost.item()
    ))

Epoch    0/20 Batch 1/3 Cost: 17185.640625
Epoch    0/20 Batch 2/3 Cost: 2274.591309
Epoch    0/20 Batch 3/3 Cost: 1493.785522
Epoch    1/20 Batch 1/3 Cost: 359.227722
Epoch    1/20 Batch 2/3 Cost: 135.375381
Epoch    1/20 Batch 3/3 Cost: 33.336632
Epoch    2/20 Batch 1/3 Cost: 13.659433
Epoch    2/20 Batch 2/3 Cost: 4.299152
Epoch    2/20 Batch 3/3 Cost: 0.017900
Epoch    3/20 Batch 1/3 Cost: 1.512465
Epoch    3/20 Batch 2/3 Cost: 1.201954
Epoch    3/20 Batch 3/3 Cost: 0.917487
Epoch    4/20 Batch 1/3 Cost: 1.302667
Epoch    4/20 Batch 2/3 Cost: 0.474953
Epoch    4/20 Batch 3/3 Cost: 1.322942
Epoch    5/20 Batch 1/3 Cost: 0.399604
Epoch    5/20 Batch 2/3 Cost: 1.046582
Epoch    5/20 Batch 3/3 Cost: 0.207082
Epoch    6/20 Batch 1/3 Cost: 0.154813
Epoch    6/20 Batch 2/3 Cost: 0.891854
Epoch    6/20 Batch 3/3 Cost: 0.879822
Epoch    7/20 Batch 1/3 Cost: 0.192756
Epoch    7/20 Batch 2/3 Cost: 0.490677
Epoch    7/20 Batch 3/3 Cost: 1.817000
Epoch    8/20 Batch 1/3 Cost: 0.662779
Epoch    

In [30]:
# 임의의 입력 [73, 80, 75]를 선언
new_var = torch.FloatTensor([[73, 80, 75]])
# 입력한 값에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var)
print('훈련 후 입력이 73, 80, 75일 때의 예측값 :', pred_y)

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[150.8657]], grad_fn=<AddmmBackward0>)
